In [1]:
import time
import math
import numpy as np
from numpy.fft  import fft2, ifft2
from scipy.signal import convolve2d
import copy

# Data Generation

In [2]:
rows = 7
cols = 7
mu, sigma = 0, math.sqrt(2.0 / (rows * cols))

k_size = 3

filter2d = np.random.normal(mu, sigma, size=(k_size, k_size))

image = np.random.randint(255, size=(rows, cols))

print("filter = ", filter2d)
print("image = ", image)

filter =  [[-0.09579081 -0.32252908 -0.0246054 ]
 [ 0.01147978  0.24092263  0.31878201]
 [-0.11307207 -0.17917608  0.21196075]]
image =  [[183 180  42 223  61  85  41]
 [107  66 139 153  77  14  95]
 [252 173  17 179 118  47  42]
 [212 191  17  47 167 199  43]
 [222 116 137 156 194 167  26]
 [208 229  51  49  91 165  27]
 [ 70  95 197 216 124 129 122]]


## Scipy Convolution

Automatically chooses direct or Fourier method based on an estimate
of which is faster (default)

In [3]:
start = time.time()
img = copy.deepcopy(image)
scipy_res = convolve2d(img, filter2d, mode='same', boundary='fill', fillvalue=0)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("scipy_res = ", scipy_res)

Finish in 2.18 ms
scipy_res =  [[ 5.32239756e+00  6.49507128e+01  8.94801102e+00  7.67186531e+00
   5.68200130e+01  2.48846475e+01  5.98956030e+00]
 [-1.24454946e+02 -1.02321812e+01  3.48104502e+01 -2.53498174e+01
   4.72468217e+01 -1.21230429e-02  2.33183678e+01]
 [-5.06083010e+01  4.88971560e+01  1.83989672e+01  1.16668918e+01
   2.89996472e+01 -1.99189368e+01 -7.71799774e+00]
 [-9.41587933e+01  7.82884750e+01  1.69218376e+01 -9.54208021e+01
  -1.34210287e+01  5.23885306e+01  6.37391949e+01]
 [-9.37877590e+01  2.52177706e+01  7.71227132e+01  3.40074012e+01
   9.56322194e+00  3.92128294e+01  8.12080262e+01]
 [-3.18298340e+01  8.16099687e+01 -1.83148316e+01 -7.81326817e+01
  -3.88042629e+01  2.09856588e+01  4.73201655e+01]
 [-4.52069647e+01  4.47537477e+01  1.14086184e+02  1.08003650e+02
   7.56363751e+01  5.86799526e+01  1.00651211e+02]]


## My implementation 1

In [4]:
def conv2d(x, f):
    r, c = x.shape
    kr, kc = f.shape
    res = np.zeros(x.shape)
    for i in range(r):
        for j in range(c):
            for mm in range(kr-1, -1, -1):
                for nn in range(kc-1, -1, -1):
                    ii = i + kr//2 - mm
                    jj = j + kc//2 - nn
                    if 0 <= ii < r and 0 <= jj < c:
                        res[i][j] += x[ii][jj] * f[mm][nn]
    return res
start = time.time()
img = copy.deepcopy(image)
res = conv2d(img, filter2d)
assert np.all((res - scipy_res) < 1e-8)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("my_res = ", res)

Finish in 1.98 ms
my_res =  [[ 5.32239756e+00  6.49507128e+01  8.94801102e+00  7.67186531e+00
   5.68200130e+01  2.48846475e+01  5.98956030e+00]
 [-1.24454946e+02 -1.02321812e+01  3.48104502e+01 -2.53498174e+01
   4.72468217e+01 -1.21230429e-02  2.33183678e+01]
 [-5.06083010e+01  4.88971560e+01  1.83989672e+01  1.16668918e+01
   2.89996472e+01 -1.99189368e+01 -7.71799774e+00]
 [-9.41587933e+01  7.82884750e+01  1.69218376e+01 -9.54208021e+01
  -1.34210287e+01  5.23885306e+01  6.37391949e+01]
 [-9.37877590e+01  2.52177706e+01  7.71227132e+01  3.40074012e+01
   9.56322194e+00  3.92128294e+01  8.12080262e+01]
 [-3.18298340e+01  8.16099687e+01 -1.83148316e+01 -7.81326817e+01
  -3.88042629e+01  2.09856588e+01  4.73201655e+01]
 [-4.52069647e+01  4.47537477e+01  1.14086184e+02  1.08003650e+02
   7.56363751e+01  5.86799526e+01  1.00651211e+02]]


## My implementation 2

In [5]:
def conv2d(x, f):
    r, c = x.shape
    kr, kc = f.shape
    res = np.zeros(x.shape)
    for i in range(r):
        for j in range(c):
            # (i, j) is the center position of filter
            for ki in range(-int(kr / 2), int(kr / 2) + 1, 1):
                for kj in range(-int(kc / 2), int(kc / 2) + 1, 1):
                    m = i - ki
                    n = j - kj
                    #print(ki, kj, i, j)
                    if m >= 0 and m < r and n >= 0 and n < c:
                        res[i, j] += x[m, n] * f[ki + int(kr / 2), kj + int(kc / 2)]
    return res
start = time.time()
img = copy.deepcopy(image)
res = conv2d(img, filter2d)
assert np.all((res - scipy_res) < 1e-8)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("my_res = ", res)

Finish in 2.13 ms
my_res =  [[ 5.32239756e+00  6.49507128e+01  8.94801102e+00  7.67186531e+00
   5.68200130e+01  2.48846475e+01  5.98956030e+00]
 [-1.24454946e+02 -1.02321812e+01  3.48104502e+01 -2.53498174e+01
   4.72468217e+01 -1.21230429e-02  2.33183678e+01]
 [-5.06083010e+01  4.88971560e+01  1.83989672e+01  1.16668918e+01
   2.89996472e+01 -1.99189368e+01 -7.71799774e+00]
 [-9.41587933e+01  7.82884750e+01  1.69218376e+01 -9.54208021e+01
  -1.34210287e+01  5.23885306e+01  6.37391949e+01]
 [-9.37877590e+01  2.52177706e+01  7.71227132e+01  3.40074012e+01
   9.56322194e+00  3.92128294e+01  8.12080262e+01]
 [-3.18298340e+01  8.16099687e+01 -1.83148316e+01 -7.81326817e+01
  -3.88042629e+01  2.09856588e+01  4.73201655e+01]
 [-4.52069647e+01  4.47537477e+01  1.14086184e+02  1.08003650e+02
   7.56363751e+01  5.86799526e+01  1.00651211e+02]]


## FFT convolution

In [6]:
from numpy.fft  import fft2, ifft2
def np_fftconvolve(A, B):
    return np.real(ifft2(fft2(A)*fft2(B, s=A.shape)))
start = time.time()
img = copy.deepcopy(image)
fft_res = np_fftconvolve(img, filter2d)
assert np.all((res - scipy_res) < 1e-8)
end = time.time()
print("Finish in %.2f ms" % ((end - start)*1000))
print("fft_res = ", res)

Finish in 71.70 ms
fft_res =  [[ 5.32239756e+00  6.49507128e+01  8.94801102e+00  7.67186531e+00
   5.68200130e+01  2.48846475e+01  5.98956030e+00]
 [-1.24454946e+02 -1.02321812e+01  3.48104502e+01 -2.53498174e+01
   4.72468217e+01 -1.21230429e-02  2.33183678e+01]
 [-5.06083010e+01  4.88971560e+01  1.83989672e+01  1.16668918e+01
   2.89996472e+01 -1.99189368e+01 -7.71799774e+00]
 [-9.41587933e+01  7.82884750e+01  1.69218376e+01 -9.54208021e+01
  -1.34210287e+01  5.23885306e+01  6.37391949e+01]
 [-9.37877590e+01  2.52177706e+01  7.71227132e+01  3.40074012e+01
   9.56322194e+00  3.92128294e+01  8.12080262e+01]
 [-3.18298340e+01  8.16099687e+01 -1.83148316e+01 -7.81326817e+01
  -3.88042629e+01  2.09856588e+01  4.73201655e+01]
 [-4.52069647e+01  4.47537477e+01  1.14086184e+02  1.08003650e+02
   7.56363751e+01  5.86799526e+01  1.00651211e+02]]


/usr/local/anaconda3/lib/python3.7/site-packages/mkl_fft/_numpy_fft.py:676: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.fftn(a, s, axes)
